In [ ]:
with open('Day07input.txt') as f:
    lines = f.readlines()

#something to remove \n 
str_lines = []
for l in lines:
    str_lines.append(l[0:-1])#''.join(lines[0:len(lines[0])-2])



# --- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:


In [ ]:
test_data = [
    "$ cd /",
    "$ ls",
    "dir a",
    "14848514 b.txt",
    "8504156 c.dat",
    "dir d",
    "$ cd a",
    "$ ls",
    "dir e",
    "29116 f",
    "2557 g",
    "62596 h.lst",
    "$ cd e",
    "$ ls",
    "584 i",
    "$ cd ..",
    "$ cd ..",
    "$ cd d",
    "$ ls",
    "4060174 j",
    "8033020 d.log",
    "5626152 d.ext",
    "7214296 k"]


The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.


In [ ]:
from anytree import Node, RenderTree
top_level = Node("/")
nodes = {}
nodes['/'] = (top_level, "dir")


In [ ]:
def run_input(elf_data):
    nodes.clear()
    nodes['/'] = (top_level, "dir")
    current_node = nodes['/'][0]
    for d in elf_data[1:]:
        if d[0] == "$":
            current_node = execute_instruction(d[2:], current_node)
        else:
            process_output(d, current_node)

def execute_instruction(d, current_node):
    if d == "ls":
        return current_node
    elif d[0:2] == "cd":
        if d[3:] == "..":
            return current_node.parent
        elif d[3:] == "/":
            return top_level
        else:
            return nodes[d[3:]][0]
        
def process_output(d, current_node):
    command = d.split()   
    newNode = Node(command[1], parent = current_node)
    nodes[command[1]] = (newNode, command[0])
                


In [ ]:
#run_input(test_data)


In [ ]:
def print_tree():
    for pre, fill, node in RenderTree(top_level):
        print("%s%s" % (pre, node.name), nodes[node.name][1])


In [ ]:

#print_tree()


Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
cd / switches the current directory to the outermost directory, /.
ls means list. It prints out all of the files and directories immediately contained by the current directory:
123 abc means that the current directory contains a file named abc with size 123.
dir xyz means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)


In [23]:
def calc_dir_size(node):
    if node[1] != "dir":
        return int(node[1])
    else: 
        sum = 0
        for c in node[0].children:
            sum = sum+calc_dir_size(nodes[c.name])
        return sum

#i = calc_dir_size(nodes["d"])
#print(i)


The total sizes of the directories above can be found as follows:


In [ ]:


#The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
#print(calc_dir_size(nodes["e"]))  

#The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
#print(calc_dir_size(nodes["a"]))  

#Directory d has total size 24933642.
#print(calc_dir_size(nodes["d"]))  


#As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
#print(calc_dir_size(nodes["/"]))  



To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)



In [35]:
def calc_sum_sub(upper_limit):
    total = 0
    for n in nodes:
        if nodes[n][1] == "dir":
            x = calc_dir_size(nodes[n])
            if x<= upper_limit:
                print(nodes[n])
                print(x)
                total = total + x
                print(total)
    return total

#print(calc_sum_sub(100000))

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

now do it with the real data

In [25]:
run_input(str_lines)
print_tree()


/
├── bzgf
│   └── htczftcn.bdr
├── dngdnvv.qdf
├── fhhwv
│   ├── bjml
│   │   ├── bzmwfgf.wwp
│   │   ├── cwghv
│   │   │   ├── htczftcn
│   │   │   ├── nflgvsgz
│   │   │   ├── svfqpfm.bgs
│   │   │   ├── tdgw
│   │   │   │   └── htczftcn.cnl
│   │   │   ├── tzrz.hms
│   │   │   └── zmm.ndc
│   │   ├── dngdnvv.qdf
│   │   ├── jbc.zgp
│   │   ├── mcqbcttc.mwr
│   │   ├── pffdg
│   │   │   ├── bbqh.wzf
│   │   │   ├── bgzdv
│   │   │   │   └── srzlfd.hnq
│   │   │   ├── bhrw.jls
│   │   │   ├── hqwmfdj
│   │   │   │   ├── dzpgcrn.rbj
│   │   │   │   ├── mrrngcdr
│   │   │   │   └── nflgvsgz.tsj
│   │   │   ├── htczftcn.lvs
│   │   │   ├── lzhz
│   │   │   │   ├── bbtdnpvf
│   │   │   │   └── jbc
│   │   │   │       └── tzrz.hms
│   │   │   ├── nbmsrl
│   │   │   │   └── dngdnvv.qdf
│   │   │   ├── qtqpw.hsj
│   │   │   └── htczftcn
│   │   │       ├── dqbml
│   │   │       │   ├── bphwp
│   │   │       │   │   ├── dngdnvv.qdf
│   │   │       │   │   ├── jsgl
│   │   │       │   │   ├──

In [26]:
print(nodes['bzgf'])

(Node('///bzgf'), 'dir')


In [36]:
xx = calc_sum_sub(100000)
print(xx)


(Node('///bzgf'), 'dir')
80802
80802
(Node('///gzlpvdhd'), 'dir')
74736
155538
(Node('///qcqg'), 'dir')
53466
209004
(Node('///fhhwv/bjml/zzvs/nflgvsgz/mdgzvzbs'), 'dir')
88913
297917
(Node('///fhhwv/bjml/zzvs/nflgvsgz/nlzc'), 'dir')
37916
335833
(Node('///fhhwv/prwpzhb/pffdg/msdtpqqd'), 'dir')
42614
378447
(Node('///fhhwv/prwpzhb/pffdg/sgtqmfgq'), 'dir')
81125
459572
(Node('///gzlpvdhd/ffvz'), 'dir')
74736
534308
(Node('///fhhwv/bjml/pffdg/htczftcn/dqbml/htczftcn/htczftcn/zzpnff'), 'dir')
79133
613441
(Node('///fhhwv/bjml/pffdg/htczftcn/dqbml/htczftcn/tpmbzslr/dbfcp'), 'dir')
23450
636891
(Node('///fhhwv/bjml/pffdg/htczftcn/dqbml/htczftcn/nflgvsgz/jbc/mzt'), 'dir')
53466
690357
(Node('///fhhwv/bjml/pffdg/htczftcn/dqbml/htczftcn/nflgvsgz/jbc/qzth/nfsbdfv/htczftcn/lnhjgl'), 'dir')
80056
770413
(Node('///fhhwv/bjml/pffdg/htczftcn/dqbml/htczftcn/nflgvsgz/svmmm/nclpsp/pwvbnc'), 'dir')
10200
780613
(Node('///fhhwv/bjml/pffdg/htczftcn/dqbml/htczftcn/nflgvsgz/svmmm/nclpsp/qhrtrsvm/qtqpw/crpzf